In [92]:
import numpy as np
import pandas as pd
import sys
from openai import OpenAI
import os
import cv2
import re
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import base64
from tqdm import tqdm
from dotenv import load_dotenv

In [ ]:
df = pd.read_csv('Data/New500.csv')
df

,id,image_name,ocr_txt
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
...,...,...,...
561,562,Meme_Data_562.jpg,Redaksi menghapus\n tulisan ini atas\n rekomen...
562,563,Meme_Data_563.jpg,Pembuat\n Sunu Dyantoro\n Aku bantu jelasin ta...
563,564,Meme_Data_564.jpg,10:19 AM\n TEMPO\n *\n Politik Politik Pendidi...
564,565,Meme_Data_565.jpg,Hehehee\n Hihihii


In [94]:
df_top10 = df.head(10)
df_top10

,id,image_name,ocr_txt
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
5,6,Meme_Data_6.jpg,"SCOOPY\n karena kamu ga pake helm,\n ga bawa S..."
6,7,Meme_Data_7.jpg,KETIKA KAMU NYARI LAGU TAPI NGGAK TAHU JUDULNY...
7,8,Meme_Data_8.jpg,PEKERJAAN PALING MUDAH ADALAH JUALAN ES BATU\n...
8,9,Meme_Data_9.jpg,DIBUTUHKAN\n KARYAWAN 3 ORANG\n PELAMAR:
9,10,Meme_Data_10.jpg,1 kerja benar kerja keras\n 2 kenangan & nama ...


In [95]:
# Check for missing values in the 'image_name' column
missing_image_names = df['ocr_txt'].isna()

# Show rows where 'image_name' is missing
df_missing = df[missing_image_names]
print(df_missing)


      id         image_name ocr_txt
287  288  Meme_Data_288.jpg     NaN
289  290  Meme_Data_290.jpg     NaN
291  292  Meme_Data_292.jpg     NaN
317  318  Meme_Data_318.jpg     NaN
336  337  Meme_Data_337.jpg     NaN
378  379  Meme_Data_379.jpg     NaN
401  402  Meme_Data_402.jpg     NaN
402  403  Meme_Data_403.jpg     NaN


In [96]:
# Drop rows where 'image_name' is missing
df = df.dropna(subset=['ocr_txt'])

# Optionally, reset the index if you want a clean index after dropping
df = df.reset_index(drop=True)

In [97]:
# Load prompt template
with open('Prompt/filter.md', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

In [98]:
# Test with first meme
test_row = df.iloc[0]
image_text = test_row['ocr_txt']
image_name = test_row['image_name']

# Replace placeholders
final_prompt = prompt_template.replace('{image_text}', str(image_text))
final_prompt = final_prompt.replace('{image_path}', image_name)

# Show the final prompt
print("FINAL PROMPT THAT WILL BE SENT TO OPENAI:")
print("="*60)
print(final_prompt)
print("="*60)

FINAL PROMPT THAT WILL BE SENT TO OPENAI:
# Role
You are a master of identifying offensive meme especially for Indonesian language

## Overview Task
•⁠  ⁠Your task is to analyze the given meme image and accompanying meme text to determine whether it contains offensive content, particularly pornographic material.
•⁠  ⁠Since the meme are fully in Indonesian language, make sure to analyze first (translate if needed by yourself).

## Content list
•⁠  ⁠Unsafe: if its contain porn reference or pornographic content whether the figure or the text
•⁠  ⁠Safe: it its not contain port reference or pornographic content whether the figure or the text

### Pornographic Content Reference
•⁠  ⁠Image that include a little kid and bunch of older people (i.e., 5 man with 1 little girl)
•⁠  ⁠Memes may use slang, metaphor, double entendre, or coded phrases to refer to sexual or pornographic content (i.e., Egg use for refering a woman chest)
•⁠  ⁠Explicitly use pornographic word (i.e., Kontol, memek, fap, co

In [99]:
def encode_image_to_base64(image_path):
    """Encode image to base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [100]:
# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from environment
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

# Verify API key is loaded
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please create a .env file with your API key.")

def classify_meme(row):
    """Classify a single meme using OpenAI Vision API"""
    # Get image path and text from your CSV structure
    image_path = f"Data/New500/{row['image_name']}"
    image_text = row['ocr_txt']
    
    # Encode image
    base64_image = encode_image_to_base64(image_path)
    
    # Create prompt by replacing placeholders
    prompt = prompt_template.replace('{image_text}', str(image_text))
    prompt = prompt.replace('{image_path}', row['image_name'])
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=10,
            temperature=0
        )
        
        result = response.choices[0].message.content.strip()
        
        # Parse result
        if "Unsafe" in result:
            classification = "Unsafe"
        elif "Safe" in result:
            classification = "Safe"
        else:
            classification = "Unclear"
            
        return {
            'classification': classification,
            'full_response': result,
            'tokens_used': response.usage.total_tokens
        }
        
    except Exception as e:
        return {
            'classification': 'Error',
            'full_response': str(e),
            'tokens_used': 0
        }

In [101]:
# Process all memes
results = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing memes"):
    result = classify_meme(row)
    results.append({
        'id': row['id'],
        'image_name': row['image_name'],
        'ocr_txt': row['ocr_txt'],
        'classification': result['classification'],
        'full_response': result['full_response'],
        'tokens_used': result['tokens_used']
    })
    
    # Optional: Add delay to respect rate limits
    import time
    time.sleep(1)

results

Processing memes: 100%|██████████| 558/558 [20:20<00:00,  2.19s/it]


[{'id': 1,
  'image_name': 'Meme_Data_1.jpg',
  'ocr_txt': 'COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK GILIRAN DIMINTAIN DUIT SALON, BILANGNYA MATRE EMANG LO KIRA PERAWATAN DI SALON PAKE BPJS??',
  'classification': 'Safe',
  'full_response': 'Output: Safe\n\nPenjelasan:  \nT',
  'tokens_used': 1172},
 {'id': 2,
  'image_name': 'Meme_Data_2.jpg',
  'ocr_txt': 'ayat suci dalam bio bukan jaminan kalo\n dia orang yang baik',
  'classification': 'Safe',
  'full_response': 'Output: Safe\n\nPenjelasan: Meme ini',
  'tokens_used': 1475},
 {'id': 3,
  'image_name': 'Meme_Data_3.jpg',
  'ocr_txt': 'Rare\n Medium Rare\n Medium\n Medium Well\n Well Done\n ONDE MANDE',
  'classification': 'Safe',
  'full_response': 'Output: Safe\n\nPenjelasan:  \nM',
  'tokens_used': 1135},
 {'id': 4,
  'image_name': 'Meme_Data_4.jpg',
  'ocr_txt': 'KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MAKSUDNYA',
  'classification': 'Safe',
  'full_response': 'Output: Safe\n\nPenjelasan: \nT',
  'tokens_used': 1141},
 

In [103]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df_unsafe = results_df[results_df['classification'] == 'Unsafe']

In [104]:
results_df_unsafe

,id,image_name,ocr_txt,classification,full_response,tokens_used
30,31,Meme_Data_31.jpg,Aduh nginjak apa nih berat banget\n Menginjak ...,Unsafe,"Unsafe\n\n**Explanation:** \nThe phrase ""Meng",1134
99,100,Meme_Data_100.jpg,"""Tertangkap saat Selingkuh dengan Janda Cantik...",Unsafe,Output: Unsafe\n\nPenjelasan: \nM,1159
233,234,Meme_Data_234.jpg,korupsi virus kontaminasi basi\n Penerima\n MB...,Unsafe,Unsafe\n\n**Reason:** \nThe image is a,1136
249,250,Meme_Data_250.jpg,ITA MARTADINATA\n Gadis keterununan Tionghoa i...,Unsafe,Output: Unsafe\n\n**Reason:** \nThe text,934
255,256,Meme_Data_256.jpg,18.26\n ll 23\n alasan tidak mau ceck hp istri...,Unsafe,Output: Unsafe\n\n**Reason:** \nThe meme,949
266,267,Meme_Data_267.jpg,Berita dan Politik\n [Happy Ending] Setelah ka...,Unsafe,Output: Unsafe\n\nPenjelasan: Terd,1192
267,268,Meme_Data_268.jpg,fufufafa\n 15-09-2014 10:16\n Pengen susunya s...,Unsafe,Output: Unsafe\n\nPenjelasan: \nT,974
268,269,Meme_Data_269.jpg,Tukang Sayur @datuakrajoa...\n Ikuti\n Kalau d...,Unsafe,Output: Unsafe\n\nPenjelasan: Teks,1189
269,270,Meme_Data_270.jpg,fufufafa\n 04-11-2013 05:17\n Quote:\n Origina...,Unsafe,Output: Unsafe\n\n**Reason:** \nThe phrase,919
270,271,Meme_Data_271.jpg,fufufafa\n 24-08-2014 23:27\n Udah dibela-bela...,Unsafe,Output: Unsafe\n\nPenjelasan: \nT,1173


In [105]:
# Save results
results_df.to_csv('meme_classification_results.csv', index=False)

# Print summary
print(f"Total processed: {len(results_df)}")
print(f"Safe: {len(results_df[results_df['classification'] == 'Safe'])}")
print(f"Unsafe: {len(results_df[results_df['classification'] == 'Unsafe'])}")
print(f"Errors: {len(results_df[results_df['classification'] == 'Error'])}")

Total processed: 558
Safe: 512
Unsafe: 46
Errors: 0
